In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from generate_data import generate_multi_sim_and_obs

from sepia.SepiaModelSetup import setup_model
from sepia.SepiaData import SepiaData

ModuleNotFoundError: No module named 'generate_data'

## Multivariate-output simple example

First, we generate the synthetic data with 50 points for each simulator run and 20 observed points. 

Why 5 basis vectors and 3 parameters $\theta_1,\theta_2,\theta_3$?

In [ ]:
seed = 42   # random seed
m = 100     # number of simulated observations
n = 1       # number of observed data
sig_n = 0.01 # observation noise SD

data_dict = generate_multi_sim_and_obs(m=m, n=n, nt_sim=50, nt_obs=20,
                                       n_theta=3, n_basis=5, 
                                       sig_n=sig_n, seed=seed)


Next, we initialize the SepiaData object which does some basic checking about whether the input data
are of the correct shapes, and infers what kind of model you're going to use based on the input data.

In [ ]:
data = SepiaData(t_sim=data_dict['t_sim'], y_sim=data_dict['y_sim'], y_ind_sim=data_dict['y_ind_sim'],
                 y_obs=data_dict['y_obs'], y_ind_obs=data_dict['y_ind_obs'])
print(data)

plt.plot(data.sim_data.y_ind, data.sim_data.y.T)
plt.plot(data.obs_data.y_ind, data.obs_data.y.T, 'k.', linewidth=3)
plt.title('Synthetic data (obs. in black)')
plt.xlabel('y index')
plt.ylabel('y')
plt.show()

Standardization of data is important for default priors in the model to work well. In this case, the y index is already on $[0,1]$. We standardize the y values as well. The function standardize y does not actually update the y values, rather it updates the mean and std. Is this because we only need to know the sufficient statistics?

We also create a PCA basis with 5 components to represent the multivariate output. Is there a way to look at the PCA to decide if 5 is the right number of basis components?

In [ ]:
data.transform_xt()
data.standardize_y()
data.create_K_basis(5)

Next, we set up the model object; a lot of precalculation of important model components is done here. We do not set up a basis for D. Is this becuase we are not modeling the discrepency in this example?

In [ ]:
model = setup_model(data)

We will use all the default priors and settings to do MCMC.

We first call `model.tune_step_sizes(50, 20)` which uses 50 samples over 20 different step sizes
to find one with a good acceptance rate.

We then draw 1000 MCMC samples.

In [ ]:
model.tune_step_sizes(50, 20)
model.do_mcmc(5000)

Here is a basic visualization of the MCMC results: histograms of the MCMC samples.

In [ ]:
# Extract MCMC samples into dictionary with parameter names
samples_dict = {p.name: p.mcmc_to_array() for p in model.params.mcmcList}

for i, k in enumerate(samples_dict.keys()):
    param_shape = samples_dict[k].shape[1]
    if param_shape >= 5:
        ncol = 5
        nrow = int(np.ceil(param_shape / ncol))
    else:
        ncol = param_shape
        nrow = 1
    plt.figure(i)
    for j in range(param_shape):
        plt.subplot(nrow, ncol, j + 1)
        #plt.hist(samples_dict[k][:, j])
        plt.plot(samples_dict[k][:, j])
        plt.xlabel(k)
    plt.show()

The parameters `betaU` and `lamUz` correspond to the Gaussian process lengthscale and marginal variance,
while `lamWs`, `lamWOs`, and `lamOs` are nugget and observation noise precisions.

Most easy to interpret is `theta`, which is the posterior distribution over the $t$ that generated $y_{obs}$.